<a href="https://colab.research.google.com/github/Rikupro-creator/LLM-prompt-recovery/blob/main/Fork_of_Fork_of_t5_small_model_7e75ab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'llm-prompt-recovery:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F67121%2F7806901%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T165012Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D595f025b03821448ee5735a47fd422d51f291b6d7ce05533a3949f22a5e2e78b4853ff48bde45fba74c0e959e9a70425901d07cb570e4def8ae5b7c0c0c98b0eb64973b42c14dac2794000eeecc10bbe25687fccf5899fcdc3b6cd758f21dee53afe3325081336db6bd3420a967e16ecd22c1b965edba6c57cbff9d05f416999ec94e0eae6ab3fa23acffa4ce4257a4742aff35dd35217a0abefa9a4ec5a40860799a1726be5bec9ceec060767169ef144d87eafda64452ac8925f45b938eb3af79eaa62df91c4e41f30b63de22b290f48874d54cf5affa4314e31c2e765ed069f6c65c10e7a7655d7f37cf4399f92b2d438857abc72a86c261cf0068c3e6573,huggingface-bert:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F934701%2F7917580%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T165012Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D328ddd938495983870fb8cd0e98ed2d7423098469088defd45a741ba99a1c6a9c2a45c4f808331cac23df7c7c6043b860941e047a9a5df63459c9f94d668a3795ae1c467ed2ad92e96858223e1968726d6bd6f1e27343138c5d12fccdf7badc557258ec05893206585d245d034c3310a9768890c77d7a6f183911f049cf5c5728efd04fe8306eec7304c704cfe995de0a3114f4ee59b56b594991c8758c52e02d9fab5fd22c40f6f63a314150b66dd0950d37598365e18edfb9261e4e53d460e7c85a9bba6d7976f134a8740860c796c840ffbf077f4a98c80fcee3d2f5dd505cf37c258041df1a611b4704482cc9e58c1e053182b5db9347b8870aab7fe6311,flan-t5/pytorch/small/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3046%2F4250%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T165012Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3f7e7b96e6b13ffd178b6d1f8d182a375badbccf504530c444f23e01e3861f9092e3c941aef192c1fc02bbce9dcf8c01067194354a22ff97e306004c8103f1b7d18a21a4a8b5f29a3ceb68915b32d8377c1c53c1c4ee880621fce5ea5d8efb8cbb457005e7c081ba8b558e89fa734b77f40385e2c17a0f2de01b7abb57c78115e50837332991abc045731cccf4e768412b546b1ce1e4fe2617514f456f333680fe2d2a29e0363468b08f9912851c323ab893fb638ebf63ab2f5d7fcc4779deddc49f30dcd7939c5570dc5d6967113baa60a17c0d93808781d2c773a1ca9aad84d2d21667a65671b2942984a0e7479f617de9a2011d28aa9b3d9f7b5a19963072,llama-2/pytorch/13b-chat-hf/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3097%2F4302%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T165012Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6373b6b30d3bc59e68fdf7569b511d6b2716d5ac9172fbebcfad07b688454f46e3ed37db0ae00bed3a50b3253be2719867c9cdf35ce7fe90be78db3b539f3bfddb6c04f8d5fa934d2c2fbc492448ce42f87d47aaedba034848c9ccbc47863ab3f2815de79bb7d464aa5951cc265d629c614424b81decc039156ae0f9eee21e8eb97636807cbd80ca650e518e1a70386640d10633d4a271036bea6456fd393dd1689c8e637c2ba640444d5a6e8997eb3057ed87402dc0a3d717137bd67225324ad74c602a21fcbe3032a02c251eb036f50d4e1f9222571c9c1b946a2a3e12ae73769e5a9943cbfc30fc5b5a979c7b8e9f538c731ef95f20f6ce9ef70cd186e119'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd


In [ ]:
test = pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')

In [ ]:
import re

def get_first_sentence(text):
    sentences = re.split('(?<=[.:!?]) +', text)
    return sentences[0]

text = "Here is your shanty: (Verse 1) The text is rewritten, ..."

first_sentence = get_first_sentence(text)

print(first_sentence)


In [ ]:
first_sentence = get_first_sentence(test['rewritten_text'][0])
first_sentence

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Define the directory path where the model and tokenizer are stored
model_directory = "/kaggle/input/flan-t5/pytorch/small/2"

# Load pre-trained T5 model and tokenizer from the local directory
tokenizer = T5Tokenizer.from_pretrained(model_directory)
model = T5ForConditionalGeneration.from_pretrained(model_directory)

question = 'question: what was provided in the text'
# Define a function to generate prompts and responses
def generate_prompts(rewritten_text):
    # Define the prompt
    prompt = f"{question}\n{rewritten_text}"
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt")
    # Generate response
    outputs = model.generate(**inputs)
    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response
# Generate responses for each text in the 'rewritten_text' column
responses = ['please improve the following text using the writing style of  ' + generate_prompts(get_first_sentence(text))+', '+', maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style.'  for text in test['rewritten_text']]


In [ ]:
responses

In [ ]:
submission=pd.DataFrame({
    'id':test.id,
    'rewrite_prompt':responses
})

In [ ]:
submission.to_csv('submission.csv', index=False)
